In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend
import matplotlib.pyplot as plt
import numpy as np

In [18]:
def model_define(drop_rate):

    model = models.Model()

    laser_input = (720,1,1)
    map_input = (428,288,1)

    model_laser = models.Sequential()
    model_laser.add(layers.Conv2D(
        32, (11,1), padding="same", input_shape=laser_input,
        name="laser_conv_1" ))
    model_laser.add(layers.Conv2D(
        64, kernel_size=(5,1), padding="same",
        name="laser_conv_2" ))
    model_laser.add(layers.MaxPooling2D(
            (2,1), strides=2, padding="same",
            name="laser_pool_1" ))
    model_laser.add(layers.Conv2D(
            256, kernel_size=(5,1), padding="same",
            name="laser_conv_3" ))
    model_laser.add(layers.MaxPooling2D(
            (2,1), strides=2, padding="same",
            name="laser_pool_2" ))
    model_laser.add(layers.Flatten())
    

    model_map = models.Sequential()
    
    model_map.add(layers.Conv2D(
            64, kernel_size=(5,1), input_shape=map_input,
            name="map_conv_2" ))
    model_map.add(layers.AveragePooling2D(
            (2,1), strides=2,
            name="map_pool_1" ))
    model_map.add(layers.Conv2D(
            256, kernel_size=(5,1),
            name="map_conv_3" ))
    model_map.add(layers.AveragePooling2D(
            (2,1), strides=2,
            name="map_pool_2" ))
    model_map.add(layers.Flatten())

    model_dense = models.Sequential()
    model_dense.add(layers.Dropout(drop_rate))
    model_dense.add(layers.Dense(1024, activation="relu", name="dense_1"))
    model_dense.add(layers.Dense(128, activation="relu", name="dense_2"))
    model_dense.add(layers.Dense(3, name="dense_3"))
    

    laser_output = model_laser(tf.ones(((1,720,1,1))))
    map_output = model_map(tf.ones((1,428,288,1)))

    cnn_output = layers.Add()([laser_output,map_output])
    
    output_ = model_dense(cnn_output)

    model = models.Model([laser_input,map_input],output_,name="model")

    return model

beta = 0.2

def pos_loss(y_actual, y_pred):
        loss_value = (1-beta)*(backend.sqrt(backend.pow((y_actual[:,0]-y_pred[:,0]),2)+backend.pow((y_actual[:,1]-y_pred[:,1]),2)))+(beta*(backend.abs(y_actual[:,2]-y_pred[:,2])))
        return loss_value

In [19]:
model = model_define(0.6)

ValueError: Inputs have incompatible shapes. Received shapes (46080,) and (1916928,)

In [ ]:
model_cnn.compile(optimizer = "adam",
    loss=pos_loss,
    metrics=['mean_absolute_error'])